In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("spark-dataframe").getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/17 09:02:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
filepath = "/home/ubuntu/working/spark-examples/data/titanic_train.csv"

# inferSchema 사용
titanic_sdf = spark.read.csv(filepath, inferSchema=True, header=True)
titanic_sdf.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [3]:
titanic_pdf = titanic_sdf.toPandas()
titanic_pdf.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


# Spark Dataframe의 컬럼, 로우(레코드) 삭제
- pandas의 데이터프레임은 `drop` 메소드를 사용. 행과 열 모두 삭제.
- spark 데이터프레임에도 `drop` 메소드를 사용. 컬럼만 삭제 가능.
    - 여러 개의 컬럼을 삭제 할 때 리스트 사용 불가
- spark에서는 데이터(row)의 삭제가 원칙적으로 불가능.
    - 데이터 삭제가 없는 대신에 `filter`를 이용해서 필요한 것만 추출. --> spark의 불변성 특성 때문

In [4]:
titanic_pdf_drop = titanic_pdf.drop("Name", axis=1)
titanic_pdf_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int32  
 1   Survived     891 non-null    int32  
 2   Pclass       891 non-null    int32  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int32  
 6   Parch        891 non-null    int32  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int32(5), object(4)
memory usage: 59.3+ KB


In [5]:
import pyspark.sql.functions as F

# drop을 사용한다고 해서 실제 삭제가 이루어지진 않는다.
# DROP으로 지정된 컬럼을 제외하고 select 한 것!
titanic_sdf_drop = titanic_sdf.drop(F.col("Name"))
titanic_sdf_drop.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [7]:
# Pclass가 1인 row를 제거 (sdf)
titanic_removed_pclass_1 = titanic_sdf.drop(F.col("Pclass") != 1)
titanic_removed_pclass_1.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

# Spark DataFrame의 dropna

In [9]:
# 레코드에 하나라도 Null 또는 NaN값이 있으면 삭제한 결과의 DataFrmae이 반환된다.
print("Dropna 이전의 행의 개수 : ", titanic_sdf.count())

titanic_sdf_dropna_1 = titanic_sdf.dropna()
print("Dropna 이후의 행의 개수 : ", titanic_sdf_dropna_1.count())

Dropna 이전의 행의 개수 :  891
Dropna 이후의 행의 개수 :  183


`dropna`를 쿼리로 표현하면?
```SQL
SELECT *
FROM titanic_sdf
WHERE Name IS NOT NULL
AND age IS NOT NULL
AND Embarked IS NOT NULL
...
```

In [10]:
# 특정 컬럼을 지정하여 거기에 null이 있는 경우만 삭제
titanic_sdf_dropna_2 = titanic_sdf.dropna(subset=["Age", "Embarked"])
titanic_sdf_dropna_2.count()

712

`dropna(subset=["Age", "Embarked"])`를 쿼리로 표현하면?
```SQL
SELECT *
FROM titanic_sdf
WHERE age IS NOT NULL
AND Embarked IS NOT NULL
```

In [12]:
# filter를 이용하여 null 제거
titanic_sdf.filter(
    F.col("Age").isNotNull() & F.col("Embarked").isNotNull()
).count()

712

In [13]:
titanic_sdf.na.drop().count()

183

In [20]:
# dropna() 메소드를 WHERE 절 로직으로 구현.
where_str = ''
column_count = len(titanic_sdf.columns)
for index, column_name in enumerate(titanic_sdf.columns):
    where_str += (column_name + ' IS NOT NULL ')
    if index < column_count -1:
        where_str += 'and '
print(where_str)

PassengerId IS NOT NULL and Survived IS NOT NULL and Pclass IS NOT NULL and Name IS NOT NULL and Sex IS NOT NULL and Age IS NOT NULL and SibSp IS NOT NULL and Parch IS NOT NULL and Ticket IS NOT NULL and Fare IS NOT NULL and Cabin IS NOT NULL and Embarked IS NOT NULL 


In [21]:
titanic_sdf.filter(where_str).count()

183

# Pandas와 Spark에서의 None, Nan, Null

In [16]:
a = None
print(type(a))

<class 'NoneType'>


In [17]:
titanic_sdf.select("Age", "Cabin").show(10)

+----+-----+
| Age|Cabin|
+----+-----+
|22.0| null|
|38.0|  C85|
|26.0| null|
|35.0| C123|
|35.0| null|
|null| null|
|54.0|  E46|
| 2.0| null|
|27.0| null|
|14.0| null|
+----+-----+
only showing top 10 rows



In [18]:
titanic_pdf[["Age", "Cabin"]].head(10)

,Age,Cabin
0,22.0,None
1,38.0,C85
2,26.0,None
3,35.0,C123
4,35.0,None
5,NaN,None
6,54.0,E46
7,2.0,None
8,27.0,None
9,14.0,None


In [19]:
titanic_sdf_from_pdf = spark.createDataFrame(titanic_pdf)
titanic_sdf_from_pdf.select("Age", "Cabin").show(10)

+----+-----+
| Age|Cabin|
+----+-----+
|22.0| null|
|38.0|  C85|
|26.0| null|
|35.0| C123|
|35.0| null|
| NaN| null|
|54.0|  E46|
| 2.0| null|
|27.0| null|
|14.0| null|
+----+-----+
only showing top 10 rows



NaN값 검색

In [22]:
titanic_sdf.filter(F.col("Age").isNull()).select("Name", "Age").show(10)

+--------------------+----+
|                Name| Age|
+--------------------+----+
|    Moran, Mr. James|null|
|Williams, Mr. Cha...|null|
|Masselmani, Mrs. ...|null|
|Emir, Mr. Farred ...|null|
|"O'Dwyer, Miss. E...|null|
| Todoroff, Mr. Lalio|null|
|Spencer, Mrs. Wil...|null|
|Glynn, Miss. Mary...|null|
|    Mamee, Mr. Hanna|null|
| Kraeff, Mr. Theodor|null|
+--------------------+----+
only showing top 10 rows



In [23]:
titanic_sdf_from_pdf.filter(F.col("Age").isNull()).select("Name", "Age").show(10)

+----+---+
|Name|Age|
+----+---+
+----+---+



In [25]:
titanic_sdf_from_pdf.filter(F.isnan(F.col("Age"))).select("Name", "Age").show(10)

+--------------------+---+
|                Name|Age|
+--------------------+---+
|    Moran, Mr. James|NaN|
|Williams, Mr. Cha...|NaN|
|Masselmani, Mrs. ...|NaN|
|Emir, Mr. Farred ...|NaN|
|"O'Dwyer, Miss. E...|NaN|
| Todoroff, Mr. Lalio|NaN|
|Spencer, Mrs. Wil...|NaN|
|Glynn, Miss. Mary...|NaN|
|    Mamee, Mr. Hanna|NaN|
| Kraeff, Mr. Theodor|NaN|
+--------------------+---+
only showing top 10 rows



# Null 컬럼명과 Null 개수 찾기

In [28]:
null_columns = [ F.col(column_name).isNull() for column_name in titanic_sdf.columns ]
null_columns

[Column<'(PassengerId IS NULL)'>,
 Column<'(Survived IS NULL)'>,
 Column<'(Pclass IS NULL)'>,
 Column<'(Name IS NULL)'>,
 Column<'(Sex IS NULL)'>,
 Column<'(Age IS NULL)'>,
 Column<'(SibSp IS NULL)'>,
 Column<'(Parch IS NULL)'>,
 Column<'(Ticket IS NULL)'>,
 Column<'(Fare IS NULL)'>,
 Column<'(Cabin IS NULL)'>,
 Column<'(Embarked IS NULL)'>]

In [29]:
titanic_sdf.select(null_columns).show(10)

+---------------------+------------------+----------------+--------------+-------------+-------------+---------------+---------------+----------------+--------------+---------------+------------------+
|(PassengerId IS NULL)|(Survived IS NULL)|(Pclass IS NULL)|(Name IS NULL)|(Sex IS NULL)|(Age IS NULL)|(SibSp IS NULL)|(Parch IS NULL)|(Ticket IS NULL)|(Fare IS NULL)|(Cabin IS NULL)|(Embarked IS NULL)|
+---------------------+------------------+----------------+--------------+-------------+-------------+---------------+---------------+----------------+--------------+---------------+------------------+
|                false|             false|           false|         false|        false|        false|          false|          false|           false|         false|           true|             false|
|                false|             false|           false|         false|        false|        false|          false|          false|           false|         false|          false|          

In [30]:
# pandas에서 누락값 개수 확인하기
titanic_pdf.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [34]:
null_count_condition = [ F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in titanic_sdf.columns]
null_count_condition

[Column<'count(CASE WHEN (PassengerId IS NULL) THEN PassengerId END) AS PassengerId'>,
 Column<'count(CASE WHEN (Survived IS NULL) THEN Survived END) AS Survived'>,
 Column<'count(CASE WHEN (Pclass IS NULL) THEN Pclass END) AS Pclass'>,
 Column<'count(CASE WHEN (Name IS NULL) THEN Name END) AS Name'>,
 Column<'count(CASE WHEN (Sex IS NULL) THEN Sex END) AS Sex'>,
 Column<'count(CASE WHEN (Age IS NULL) THEN Age END) AS Age'>,
 Column<'count(CASE WHEN (SibSp IS NULL) THEN SibSp END) AS SibSp'>,
 Column<'count(CASE WHEN (Parch IS NULL) THEN Parch END) AS Parch'>,
 Column<'count(CASE WHEN (Ticket IS NULL) THEN Ticket END) AS Ticket'>,
 Column<'count(CASE WHEN (Fare IS NULL) THEN Fare END) AS Fare'>,
 Column<'count(CASE WHEN (Cabin IS NULL) THEN Cabin END) AS Cabin'>,
 Column<'count(CASE WHEN (Embarked IS NULL) THEN Embarked END) AS Embarked'>]

In [35]:
titanic_sdf.select(null_count_condition).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



## null 값 처리하기

In [36]:
# pandas 데이터 프레임에서 age 컬럼의 null 값을 age의 평균으로 채우기
titanic_pdf["Age"] = titanic_pdf['Age'].fillna(titanic_pdf["Age"].mean())
titanic_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int32  
 1   Survived     891 non-null    int32  
 2   Pclass       891 non-null    int32  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int32  
 7   Parch        891 non-null    int32  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int32(5), object(5)
memory usage: 66.3+ KB


In [37]:
# 스칼라 값으로 NaN의 값을 채웠음.
titanic_pdf["Age"].mean()

29.69911764705882

In [39]:
# 스파크 데이터프레임에서 결측치 채우기
titanic_sdf.fillna(value=999).select("Age", "Cabin").show(10)
titanic_sdf.fillna(value='NA').select("Age", "Cabin").show(10)

+-----+-----+
|  Age|Cabin|
+-----+-----+
| 22.0| null|
| 38.0|  C85|
| 26.0| null|
| 35.0| C123|
| 35.0| null|
|999.0| null|
| 54.0|  E46|
|  2.0| null|
| 27.0| null|
| 14.0| null|
+-----+-----+
only showing top 10 rows

+----+-----+
| Age|Cabin|
+----+-----+
|22.0|   NA|
|38.0|  C85|
|26.0|   NA|
|35.0| C123|
|35.0|   NA|
|null|   NA|
|54.0|  E46|
| 2.0|   NA|
|27.0|   NA|
|14.0|   NA|
+----+-----+
only showing top 10 rows



In [40]:
# Age에 대한 결측치를 Age의 평균으로 처리

mean_age = titanic_sdf.select(F.avg(F.col("Age")))
mean_age.show()

+-----------------+
|         avg(Age)|
+-----------------+
|29.69911764705882|
+-----------------+



In [41]:
type(mean_age)

pyspark.sql.dataframe.DataFrame

In [42]:
titanic_sdf.fillna(value=mean_age, subset=["Age"])

TypeError: value should be a float, int, string, bool or dict

In [43]:
# 데이터 프레임을 RDD 조회 결과로 확인
mean_avg_row = mean_age.collect()
mean_avg_row

[Row(avg(Age)=29.69911764705882)]

In [46]:
row = mean_avg_row[0] # 첫 번째 로우 가져오기
mean_age_value = row[0] # 첫 번째 컬럼 가져오기
mean_age_value

29.69911764705882

In [47]:
titanic_sdf.fillna(value=mean_age_value, subset=["Age"]).select("Age").show()

+-----------------+
|              Age|
+-----------------+
|             22.0|
|             38.0|
|             26.0|
|             35.0|
|             35.0|
|29.69911764705882|
|             54.0|
|              2.0|
|             27.0|
|             14.0|
|              4.0|
|             58.0|
|             20.0|
|             39.0|
|             14.0|
|             55.0|
|              2.0|
|29.69911764705882|
|             31.0|
|29.69911764705882|
+-----------------+
only showing top 20 rows



In [48]:
spark.stop()